In [16]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
df  = pd.read_csv("temperature_daily.csv")
df["date"] = pd.to_datetime(df["date"], errors="raise")  # or errors="coerce"
# print(df["date"].dtype)
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
#keeping only the last 10 years

df_mask = (df['date'] >= "2008-01-01") & (df['date'] <= "2017-12-31")

df_last_10 = df.loc[df_mask].copy()


In [14]:
def get_month(df, month, year):
    mask = (df["year"] == year) & (df["month"] == month)
    return df.loc[mask].copy()

In [23]:
# testing
months = [1, 2]
years = [2014, 2015]

subset = df[df["year"].isin(years) & df["month"].isin(months)].copy()

fig = make_subplots(
    rows=2, cols=2, shared_xaxes=False, shared_yaxes=False,
    horizontal_spacing=0.05, vertical_spacing=0.10,
    subplot_titles=[
        f"{years[0]}-Jan", f"{years[1]}-Jan",
        f"{years[0]}-Feb", f"{years[1]}-Feb",
    ]
)


pos = {
    (years[0], 1): (1, 1),
    (years[1], 1): (1, 2),
    (years[0], 2): (2, 1),
    (years[1], 2): (2, 2),
}

for (y, m), (r, c) in pos.items():
    sub = subset[(subset["year"] == y) & (subset["month"] == m)]
    if sub.empty:
        continue

    # background "tile" using max temp as color (we'll later add min toggle)
    fig.add_trace(
        go.Heatmap(
            z=[[sub["max_temperature"].mean()]],  # 1×1 tile
            x=[0], y=[0],
            colorscale="Turbo",
            showscale=False,
            hovertemplate=f"Year {y}, month {m}<br>avg max %{sub['max_temperature'].mean():.1f} °C<extra></extra>",
        ),
        row=r, col=c,
    )

    # line for daily max temperature inside the tile
    fig.add_trace(
        go.Scatter(
            x=sub["date"],
            y=sub["max_temperature"],
            mode="lines",
            line=dict(color="white", width=1),
            hovertemplate="Date %{x|%Y-%m-%d}<br>Max %{y} °C<extra></extra>",
            showlegend=False,
        ),
        row=r, col=c,
    )

    # optionally hide axes for a “mini chart” look
    fig.update_xaxes(visible=False, row=r, col=c)
    fig.update_yaxes(visible=False, row=r, col=c)

# ---- 4. Layout and show / save ----
fig.update_layout(
    title="Prototype 2×2 temperature grid",
    height=600,
    width=800,
)

fig.write_html("grid_demo.html", auto_open=True)       
